In [ ]:
import pandas as pd
import numpy as np
import os
import glob

from datetime import datetime

In [ ]:
filename = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Current_G3x_Comm.csv'
df = pd.read_csv(filename, sep = ',')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('#', 'num')

df['Net_Shipments'] = df['Net_Shipments'].str.replace(',','')
df['Tier_1_Objective'] = df['Tier_1_Objective'].str.replace(',','')
df['Tier_2_Objective'] = df['Tier_2_Objective'].str.replace(',','')
df['Tier_3_Objective'] = df['Tier_3_Objective'].str.replace(',','')

filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/GDY_Comm_Align_Incentive.txt'
df_tierMap_align = pd.read_csv(filename2, sep = '\t')

filename3 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/GDY_Comm_Base_Incentive.txt'
df_tierMap_base = pd.read_csv(filename3, sep = '\t')
df.shape

In [ ]:
df_select = df[
    ['G3X_CO_Name','Aligned','Net_Shipments','Tier_1_Objective','Tier_2_Objective','Tier_3_Objective',\
     'Goodyear_Core_Net_Shipments', 'Goodyear_Mid_Net_Shipments', 'Kelly_Net_Shipments','Last_Transaction']]

In [ ]:
currentYear = datetime.now().year

In [ ]:
tup_now = datetime.now().timetuple()

In [ ]:
ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData)
# print(dt_currentData)
ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_incentive)

In [ ]:
calculatedData = pd.DataFrame([])
for ii in range(df.shape[0]):

    singleCusty = df_select.iloc[[ii]]
    singleCusty.reset_index(inplace=True)

    tt = pd.to_datetime(singleCusty['Last_Transaction'])[0].timetuple()
        
    if  tt.tm_year == currentYear:

        if (tt.tm_mon == 1) | (tt.tm_mon == 2) | (tt.tm_mon == 3):
            if ((tt.tm_year % 400 == 0) | (tt.tm_year % 4 == 0)) & (tt.tm_year % 100 != 0):
                quarter = 1
                daysINqtr = 91
                percentPayout = 0.25
                print('Leap year')
            else:
                quarter = 1
                daysINqtr = 90
                percentPayout = 0.25
        elif (tt.tm_mon == 4) | (tt.tm_mon == 5) | (tt.tm_mon == 6):
            quarter = 2
            daysINqtr = 91
            print(quarter)
            break # for testing            
        elif (tt.tm_mon == 7) | (tt.tm_mon == 8) | (tt.tm_mon == 9):
            quarter = 3
            daysINqtr = 92
            print(quarter)
            break # for testing
        else:
            quarter = 4
            daysINqtr = 92
            print(quarter)
            break # for testing
        
        numDays = tt.tm_yday # get number of days

        estimatedTires = np.floor(singleCusty.Net_Shipments.astype(int)/(numDays/daysINqtr)).values
        netShip = singleCusty.Net_Shipments.values.astype(int)
        netGYc = singleCusty.Goodyear_Core_Net_Shipments.values.astype(int)
        netGYm = singleCusty.Goodyear_Mid_Net_Shipments.values.astype(int)
        netKelly = singleCusty.Kelly_Net_Shipments.values.astype(int)
        
        objective1 = singleCusty.Tier_1_Objective.values.astype(int)
        objective2 = singleCusty.Tier_2_Objective.values.astype(int)
        objective3 = singleCusty.Tier_3_Objective.values.astype(int)

        # Current Tier
        if   (netShip >= objective1) & (netShip < objective2):
            currentTier = 1
        elif (netShip >= objective2) & (netShip < objective3):
            currentTier = 2
        elif (netShip >= objective3):
            currentTier = 3
        else:
            currentTier = 0
        
        # Estimated Tier
        if   (estimatedTires >= objective1) & (estimatedTires < objective2):
            estimatedTier = 1
            objective = objective2
        elif (estimatedTires >= objective2) & (estimatedTires < objective3):
            estimatedTier = 2
            objective = objective3
        elif (estimatedTires >= objective3):
            estimatedTier = 3
            objective = objective3
        else:
            estimatedTier = 0
            objective = objective1
            
        # Up Tier
        if estimatedTier == 3:
            upTier = 3
        else:
            upTier = estimatedTier + 1
        
        # Check if customer is aligned
        if singleCusty.Aligned.values == 'N':
            df_tierMap = df_tierMap_base
        else:
            df_tierMap = df_tierMap_align

        # floor to underestimate estimated tires
        goodYR_core_est_tires = np.floor(netGYc/(numDays/daysINqtr))
        goodYR_core_est_tires_rebate = np.floor(goodYR_core_est_tires * df_tierMap.iloc[estimatedTier,1] * percentPayout)
        goodYR_core_prop = netGYc/netShip
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
        goodYR_core_1up_tires = np.ceil(goodYR_core_prop*objective)
        goodYR_core_1up_tires_rebate = np.floor(goodYR_core_1up_tires * df_tierMap.iloc[upTier,1] * percentPayout)

        goodYR_mid_est_tires = np.floor(netGYm/(numDays/daysINqtr))
        goodYR_mid_est_tires_rebate = np.floor(goodYR_mid_est_tires * df_tierMap.iloc[estimatedTier,2] * percentPayout)
        goodYR_mid_prop = netGYm/netShip
        goodYR_mid_1up_tires = np.ceil(goodYR_mid_prop*objective)
        goodYR_mid_1up_tires_rebate = np.floor(goodYR_mid_1up_tires * df_tierMap.iloc[upTier,2] * percentPayout)
        

        kelly_est_tires = np.floor(netKelly/(numDays/daysINqtr))
        kelly_est_tires_rebate = np.floor(kelly_est_tires * df_tierMap.iloc[estimatedTier,3] * percentPayout)
        kelly_prop = netKelly/netShip
        kelly_1up_tires = np.ceil(kelly_prop*objective)
        kelly_1up_tires_rebate = np.floor(kelly_1up_tires * df_tierMap.iloc[upTier,3] * percentPayout)
        
        total_est_tires = goodYR_core_est_tires + goodYR_mid_est_tires + kelly_est_tires
        total_est_dollars = np.floor((goodYR_core_est_tires_rebate+goodYR_mid_est_tires_rebate+kelly_est_tires_rebate) * percentPayout)
        total_1up_dollars = np.floor((goodYR_core_1up_tires_rebate+goodYR_mid_1up_tires_rebate+kelly_1up_tires_rebate) * percentPayout)
        total_1up_tires = goodYR_core_1up_tires+goodYR_mid_1up_tires+kelly_1up_tires
        
        if estimatedTires > total_1up_tires:
            goodYR_core_1up_tires = goodYR_core_est_tires
            goodYR_mid_1up_tires = goodYR_mid_est_tires
            kelly_1up_tires = kelly_est_tires
            total_1up_tires = estimatedTires
            
            goodYR_core_1up_tires_rebate = goodYR_core_est_tires_rebate
            goodYR_mid_1up_tires_rebate = goodYR_mid_est_tires_rebate
            kelly_1up_tires_rebate = kelly_est_tires_rebate
            total_1up_dollars = total_est_dollars

        range_dollars_ll = np.floor((total_1up_tires*df_tierMap.iloc[upTier,3]*percentPayout))#.values
        range_dollars_ul = np.floor((total_1up_tires*df_tierMap.iloc[upTier,1]*percentPayout))#.values
        
###########################################################################################################################        
#         print(singleCusty)
# ###########################################################################################################################     
#         print('\nEstimated Tier:', [estimatedTier])
        
#         print('\nEstimated Tires ~ Goodyear:', goodYR_core_est_tires)
#         print('Estimated Tires ~ Kelly:', goodYR_mid_est_tires)        
#         print('Estimated Tires ~ Dunlop:', kelly_est_tires)
#         print('-------------------------------------')
#         print('Estimated Tires ~ Total:',estimatedTires)
        
#         print('\nEstimated Dollars ~ Goodyear:',goodYR_core_est_tires_rebate)
#         print('Estimated Dollars ~ Kelly:',goodYR_mid_est_tires_rebate)
#         print('Estimated Dollars ~ Dunlop:',kelly_est_tires_rebate)
#         print('-------------------------------------')
#         print('Estimated Dollars ~ Total:',total_est_dollars)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################
#         print('Estimated 1up Tier:', [upTier])
        
#         print('\n1up Tires ~ Goodyear:', goodYR_core_1up_tires)
#         print('1up Tires ~ Kelly:', goodYR_mid_1up_tires)        
#         print('1up Tires ~ Dunlop:', kelly_1up_tires)
#         print('-------------------------------------')
#         print('Estimated 1up Tires ~ Total:',total_1up_tires)
        
#         print('\n1up Dollars ~ Goodyear:',goodYR_core_1up_tires_rebate)
#         print('1up Dollars ~ Kelly:',goodYR_mid_1up_tires_rebate)
#         print('1up Dollars ~ Dunlop:',kelly_1up_tires_rebate)
#         print('-------------------------------------')
#         print('1up Dollars ~ Total:',total_1up_dollars)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################        
#         print('Range_ll:',range_dollars_ll)
#         print('Range_ul:',range_dollars_ul, '\n')
# ###########################################################################################################################
#         print(tt, '\n' 'indexNumber:', ii, '\n')
###########################################################################################################################

        tempData = pd.DataFrame(
            {'estimated_tier':estimatedTier,
             'GY_core_est_tires':goodYR_core_est_tires, 'GY_mid_est_tires':goodYR_mid_est_tires, 'kelly_est_tires':kelly_est_tires, 'current_est_tires':total_est_tires,
             'GY_core_est_rebate':goodYR_core_est_tires_rebate, 'GY_mid_est_rebate':goodYR_mid_est_tires_rebate, 'kelly_est_rebate':kelly_est_tires_rebate,
             'total_est_rebate': total_est_dollars,'Up_tier':upTier,
             'GY_core_up_tires':goodYR_core_1up_tires, 'GY_mid_up_tires':goodYR_mid_1up_tires, 'kelly_up_tires':kelly_1up_tires, 'total_up_tires':total_1up_tires,
             'GY_core_up_rebate':goodYR_core_1up_tires_rebate, 'GY_mid_up_rebate':goodYR_mid_1up_tires_rebate, 'kelly_up_rebate':kelly_1up_tires_rebate,
             'total_up_rebate':total_1up_dollars,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul,'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
    else:
#         print('\nJUNK:', tt,'\n', 'indexNumber:', ii)
        range_dollars_ll = (df_tierMap.iloc[1,3] * objective)
        range_dollars_ul = (df_tierMap.iloc[1,1] * objective) 
        tempData = pd.DataFrame(
            {'estimated_tier':0,
             'GY_core_est_tires':0, 'GY_mid_est_tires':0, 'kelly_est_tires':0, 'current_est_tires':0,
             'GY_core_est_rebate':0, 'GY_mid_est_rebate':0, 'kelly_est_rebate':0,
             'total_est_rebate': 0,'Up_tier':1,
             'GY_core_up_tires':0, 'GY_mid_up_tires':0, 'kelly_up_tires':0, 'total_up_tires':0,
             'GY_core_up_rebate':0, 'GY_mid_up_rebate':0, 'kelly_up_rebate':0,
             'total_up_rebate':0,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul,'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
#         print(ii)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df], axis=1)

In [ ]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_G3X_COMM.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_G3X_COMM_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

In [ ]:
finalData.head(1)

In [ ]:
# path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
# writeFilename = path + 'cmp_G3X.csv'
# finalData.to_csv(writeFilename, index=False)
# writeFilename_archive = path + 'csv_archive/' + str(dt_currentData.year) + '_' + str(dt_currentData.month) + '_' + str(dt_currentData.day) + '_' + 'CMP_G3X.csv'
# finalData.to_csv(writeFilename_archive, index=False)